# FILE ORGANIZER

___

## 1. Organizzazione dei file

In questo notebook si documenterà la creazione di uno script per organizzare i file contenuti nella cartella "files".
Durante l'esecuzione lo script mostrerà log tipo:
```
bw type:image size:94926B
ciao type:doc size:12B
daffodil type:image size:24657B
eclipse type:image size:64243B
pippo type:doc size:8299B
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B
```

I file verranno organizzati in sottocartelle della cartella "files" con una struttura tipo:
```
- files
    - audio
        - song1.mp3
        - song2.mp3
    - docs
        - ciao.txt
        - pippo.odt
    - images
        - bw.png
        - daffodil.jpg
        - eclipse.png
        - trump.jpeg
    - recap.csv
```


Il file recap.csv contine i log creati dallo script.

___

### 1.1 Script per organizzare i file

In [1]:
# Librerie necessarie
import os
import magic
import re
import shutil
import csv

In [2]:
# Funzione generatore che permette di iterare tutti i file nella cartella file, senza considerare il file recap.csv
def get_files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path,file)) and file != 'recap.csv':
            yield file

In [3]:
# Funzione per ottenere il tipo di file
def get_type(file):
    f = magic.Magic(mime=True)
    file_type = f.from_file(os.path.join(files_path, file))
    if re.search(r'audio', file_type):
        return 'audio'
    elif re.search(r'text', file_type):
        return 'doc'
    elif re.search(r'image', file_type):
        return 'image'
    else:
        raise TypeError(f"{file}: Tipo di file non supportato!")

In [4]:
# Funzione per ottenere la dimensione di un file
def get_size(file):
    return os.path.getsize(os.path.join(files_path, file))

In [5]:
# Funzione per ottenere il nome del file senza estensione
def get_name(file):
    file = os.path.splitext(file)[0]
    return file.split('/')[-1]

In [6]:
# Ottengo il percorso della cartella contenente i file da organizzare
files_directory = "files"

files_path = os.path.join(os.getcwd(), files_directory)

In [7]:
# Controllo se sono presenti le sottocartelle. Se non sono presenti le creo.
sub_directories = ['audio', 'docs', 'images']

for dir in sub_directories:
    if not os.path.isdir(os.path.join(files_path, dir)):
        os.mkdir(os.path.join(files_path, dir))

In [8]:
# Loop principale
done = False
while not done:
    # Controlla se il file recap.csv esiste
    if os.path.isfile(os.path.join(files_path, 'recap.csv')):
        recap_csv = open(os.path.join(files_path, 'recap.csv'), "a") # se esiste si apre in modalità "append" per non sovrascrivere il file esistente
        writer = csv.writer(recap_csv) # Oggetto "writer" per aggiungere righe al file
    else:
        recap_csv = open(os.path.join(files_path, 'recap.csv'), "x") # se non esiste si crea in modalità esclusiva (restituisce errore se il file esiste)
        title_row = ['name', 'type', 'size(B)'] # Prima riga con i nomi delle colonne
        writer = csv.writer(recap_csv) # Oggetto "writer" per aggiungere righe al file
        writer.writerow(title_row) # Inizializza il file csv con i titoli delle colonne
    
    for file in get_files(files_path):
        print(f"{get_name(file)} type:{get_type(file)} size:{get_size(file)}B") # Stampa a schermo l'operazione
        row = [get_name(file), get_type(file), get_size(file)] # Salva l'operazione su csv
        writer.writerow(row)
        # Sposta i file in base al tipo
        if get_type(file) == 'audio':
            shutil.move(os.path.join(files_path, file), os.path.join(files_path, 'audio'))
        elif get_type(file) == 'doc':
            shutil.move(os.path.join(files_path, file), os.path.join(files_path, 'docs'))
        else:
            shutil.move(os.path.join(files_path, file), os.path.join(files_path, 'images'))

    recap_csv.close() # Chiude il file csv
    done = True # Interrompe il loop

___

## 2. Conversione dello script da notebook a eseguibile

Lo script realizzato nel capitolo 1 è stato convertito in eseguibile. L'eseguibile nominato `addfile.py` si trova nella stessa cartella del notebook.

L'eseguibile è stato realizzato utilizzando la libreria `typer` in quanto essa fornisce automaticamente messaggi di errore per argomenti mancanti e funzione help.

Il file da organizzare si deve trovare all'interno della cartella `files` ma **NON** in una delle sottocartelle.

Lo script stamperà a schermo i dati sul file che verranno poi scritti in `recap.csv`.

___

## 3. Utilizzo di NumPy e PIL per mostrare informazionni sulle immagini

In [9]:
from PIL import Image
import numpy as np
from rich.console import Console
from rich.table import Table

image_path = os.path.join(files_path, sub_directories[2]) # Cartella contenente le immagini

# Inizializzo la tabella
table = Table(title="Images")
table_titles = ["Name","Height","Width","Grayscale","Red","Green","Blue","ALPHA"]
for title in table_titles:
    table.add_column(title)
# Creo una nuova riga nella tabella per ogni immagine
for image_file in os.listdir(image_path):
    with Image.open(os.path.join(image_path, image_file)) as image:
        image_arr = np.array(image) # Converto l'immagine in array
        if image_arr.ndim == 2: # Se l'immagine è in bianco e nero
            table.add_row(
                str(get_name(image_file)), # Nome del file
                str(image_arr.shape[0]), # Altezza
                str(image_arr.shape[1]), # Larghezza
                str(np.around(image_arr.mean(dtype=np.float64), decimals=2)), # Scala di grigi
                "0.00", # Rosso
                "0.00", # Verde
                "0.00", # Blu
                "0.00" # Alpha
            )
        elif image_arr.shape[2] == 3: # Se l'immagine è a colori senza alpha
            table.add_row(
                str(get_name(image_file)), # Nome del file
                str(image_arr.shape[0]), # Altezza
                str(image_arr.shape[1]), # Larghezza
                "0.00", # Scala di grigi
                str(np.around(np.mean(image_arr[:,:,0]), decimals=2)), # Rosso
                str(np.around(np.mean(image_arr[:,:,1]), decimals=2)), # Verde
                str(np.around(np.mean(image_arr[:,:,2]), decimals=2)), # Blu
                "0.00" # Alpha
            )
        else: # Se l'immagine è a colori con alpha
            table.add_row(
                str(get_name(image_file)), # Nome del file
                str(image_arr.shape[0]), # Altezza
                str(image_arr.shape[1]), # Larghezza
                "0.00", # Scala di grigi
                str(np.around(np.mean(image_arr[:,:,0]), decimals=2)), # Rosso
                str(np.around(np.mean(image_arr[:,:,1]), decimals=2)), # Verde
                str(np.around(np.mean(image_arr[:,:,2]), decimals=2)), # Blu
                str(np.around(np.mean(image_arr[:,:,3]), decimals=2)) # Alpha
            )

console = Console()
console.print(table)


                                   Images                                   
┏━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┓
┃ Name     ┃ Height ┃ Width ┃ Grayscale ┃ Red    ┃ Green  ┃ Blue  ┃ ALPHA  ┃
┡━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━┩
│ daffodil │ 500    │ 335   │ 0.00      │ 109.23 │ 85.52  │ 4.77  │ 0.00   │
│ trump    │ 183    │ 275   │ 0.00      │ 97.01  │ 98.99  │ 90.92 │ 0.00   │
│ bw       │ 512    │ 512   │ 21.48     │ 0.00   │ 0.00   │ 0.00  │ 0.00   │
│ eclipse  │ 256    │ 256   │ 0.00      │ 109.05 │ 109.52 │ 39.85 │ 133.59 │
└──────────┴────────┴───────┴───────────┴────────┴────────┴───────┴────────┘